In [1]:
import warnings
import os
import pandas as pd
from pathlib import Path
warnings.filterwarnings('ignore')


doc_name = "因子分析阶段表格数据"
base_path = Path(os.path.abspath('.')) / 'assert'
# 是否筛选数字化程度高一点的数据
# dataset = dataset[(dataset["wufei"]>0.2)&(dataset["zhaochenyv"]>0.5)&(dataset["yuanchun"]>0.5)&(dataset["lishouxi"]>0.5)&(dataset["csmar"]>10)]
dataset = pd.read_csv('./data/制造企业绩效评价数据总表.csv', dtype={"股票代码": 'object'})
dataset = dataset[dataset['截止日期']==2023].reset_index(drop=True)
dataset.drop(["截止日期","wufei", "zhaochenyv", "yuanchun", "lishouxi", "csmar"], axis=1, inplace=True)
print(f"筛选保留2023年数据{dataset.shape[0]}个。")

筛选保留2023年数据1515个。


In [2]:
import numpy as np
from utils.config import EvaluationIndicatorTable, pos_indicators
from utils.config import table_translate
from factor_analyzer.factor_analyzer import calculate_kmo, calculate_bartlett_sphericity

data = dataset.copy(deep=True)
# 将构建的指标体系转化为表格形式
ind_table = pd.DataFrame(EvaluationIndicatorTable)
table_translate(ind_table, filename=doc_name, table_name="评价指标体系表")
for name in pos_indicators:
    max_ = data[name].max()
    data[name] = data[name].apply(lambda x: max_ - x)

un_unit = data[["股票代码", "股票简称", '行业代码', '所属省份', "股权性质"]].set_index("股票简称")
data = data.set_index("股票简称")[ind_table["指标层"].tolist()].astype('float')

mo_all, kmo_model = calculate_kmo(data)
chi_square, p = calculate_bartlett_sphericity(data)
print(f"KMO统计量的值为：{kmo_model:.4f}")
print(f"在{p:.3f}的显著水平下，近似卡方{chi_square:.3f}，球形检验拒绝相关阵为单位阵的原假设，说明做因子分析的效果还可以。")

KMO统计量的值为：nan
在nan的显著水平下，近似卡方nan，球形检验拒绝相关阵为单位阵的原假设，说明做因子分析的效果还可以。


In [ ]:
from factor_analyzer.factor_analyzer import FactorAnalyzer

# 首先确定需要保留多少个主成分
fa = FactorAnalyzer(n_factors=data.shape[1], rotation=None, method='principal')
fa.fit(data)
# 依次计算各类方差贡献率
eig_df = pd.DataFrame(fa.get_eigenvalues()[1], columns=["总计"])
# eig_df = eig_df.sort_values("总计", ascending=False)
eig_df['方差贡献率(%)'] = eig_df["总计"].apply(lambda x: x * 100 / sum(eig_df["总计"]))
eig_df['累积贡献率(%)'] = eig_df['方差贡献率(%)'].cumsum()

In [ ]:
print(fa.get_eigenvalues()[1])
# 因子分析建模: 最大方差旋转
n_factors = 7
fa = FactorAnalyzer(n_factors=n_factors, rotation='varimax', method='principal')
fa.fit(data)

# 旋转后解析结果
eig_table = eig_df[:n_factors].copy(deep=True)
eig_table["总计2"] = np.sum(np.square(fa.loadings_), axis=0)
eig_table["方差贡献率2(%)"] = eig_table["总计2"].apply(lambda x: x / eig_df["总计"].sum() * 100)
eig_table["累积贡献率2(%)"] = eig_table["方差贡献率2(%)"].cumsum()
eig_table = eig_table.round(3)
eig_table.columns = [i for i in range(6)]
eig_table.loc[-1] = ["总计", "方差贡献率(%)", "累积贡献率(%)", "总计", "方差贡献率(%)", "累积贡献率(%)"]
eig_table.loc[-2] = ["提取载荷平方和", "提取载荷平方和", "提取载荷平方和", "旋转载荷平方和", "旋转载荷平方和",
                     "旋转载荷平方和"]
eig_table.index += 2
eig_table.sort_index(inplace=True)
table_translate(eig_table, filename=doc_name, table_name="方差解释率表")
eig_table

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = 'Times New Roman'
plt.rcParams['axes.unicode_minus'] = False

alpha = 0.8
linewidth = 2
color = 'grey'
marker = "o"
markersize = 5
markeredgecolor = 'k'
markeredgewidth = 1
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(10, 3.6), dpi=300)

# font = {"fontsize":10, "fontfamily": "Songti SC"}  
font = {"fontsize": 10, "fontfamily": "simsun"}
names = range(1, eig_df.shape[0] + 1)
ax1.set_title("碎石图", fontdict=font, fontweight='heavy')
ax1.plot(names, eig_df['总计'], linestyle='-', linewidth=linewidth, color=color, marker=marker,
         markeredgecolor=markeredgecolor, alpha=alpha, markersize=markersize, markeredgewidth=markeredgewidth)
# ax1.axhlineine(y=eig_df['总计'].sum()/len(eig_df['总计'])+0.1, xmin=-1.5, xmax=10, color='black', linestyle=':')
ax1.set_xlabel("主成分号", fontdict=font)
ax1.set_ylabel("特征值", fontdict=font)
ax1.set_xticks(range(eig_df.shape[0] + 1)[::5])
ax1.tick_params(direction='out', length=4, width=1, bottom=True, left=True)
ax1.set_xlim(0, eig_df.shape[0] + 1)
ax1.set_yticks([0, 1, 2, 3, 4, 5, 6, 7])
ax1.grid(linestyle="--", lw=0.5, color="#4E616C")
for spine in ax1.spines.values():
    spine.set_linewidth(1)
    spine.set_edgecolor("black")

ax2.set_title("方差解释及累计方差贡献率图", fontdict=font, fontweight='heavy')
ax2.plot(names, eig_df['方差贡献率(%)'], linestyle='-', linewidth=linewidth, color=color, marker=marker,
         markeredgecolor=markeredgecolor, alpha=alpha, markersize=markersize, markeredgewidth=markeredgewidth)
ax2.plot(names, eig_df['累积贡献率(%)'], linestyle='-.', linewidth=linewidth, color=color, marker=marker,
         markeredgecolor=markeredgecolor, alpha=alpha, markersize=markersize, markeredgewidth=markeredgewidth)
ax2.set_yticks([0, 20, 40, 60, 80, 100])
ax2.tick_params(direction='out', length=4, width=1, bottom=True, left=True)
ax2.set_xlabel("主成分号", fontdict=font)
ax2.set_ylabel("方差解释率(%)", fontdict=font)
ax2.set_xticks(range(eig_df.shape[0] + 1)[::5])
ax2.grid(linestyle="--", lw=0.5, color="#4E616C")
for spine in ax2.spines.values():
    spine.set_linewidth(1)
    spine.set_edgecolor("black")
plt.savefig(base_path / 'imgs/碎石图.svg', bbox_inches='tight', pad_inches=0.0, transparent=True)
plt.show()

In [ ]:
import seaborn as sns

sns.set(font='Times New Roman')

# font = {"fontfamily":"Songti SC", "fontsize":10}  
font = {"fontsize": 10, "fontfamily": "simsun"}
fig = plt.figure(figsize=(6, 6), dpi=300)
ax = fig.add_axes([0, 0, 1, 1])
sns.heatmap(np.abs(fa.loadings_), cmap='Blues', annot=True, fmt='.3f', ax=ax,
            annot_kws={"fontfamily": "Times New Roman", "fontsize": 12},
            xticklabels=[f'F{i + 1}' for i in range(n_factors)], yticklabels=data.columns)
ax.set_xlabel('主因子', fontdict=font)
ax.set_ylabel('评价指标', fontdict=font)
ax.set_yticklabels(ax.get_yticklabels(), fontdict=font)
plt.tight_layout()
plt.savefig(base_path / 'imgs/旋转后的载荷矩阵.svg', transparent=True, pad_inches=0.0, bbox_inches='tight')
plt.show()

In [ ]:
# 计算最终因子得分-形成训练数据

# aa1 = fa.transform(matrix).dot(eig_df['方差贡献率(%)'][:n_factors].to_numpy())
# weights_ = np.linalg.solve(fa.corr_, fa.loadings_)
# X_scale = (metrix - np.mean(metrix, axis=0)) / np.std(metrix, axis=0)
# aa2 = X_scale.dot(weights_.dot(eig_df['方差贡献率(%)'][:n_factors].to_numpy()))
# weights_.shape
# _w = weights_.dot(eig_df['方差贡献率(%)'][:n_factors].to_numpy())

# _data["因子得分"] = fa.transform(data).dot(eig_df['方差贡献率(%)'][:n_factors].to_numpy())
# _data = pd.merge(un_unit, _data, left_index=True, right_index=True)
# _data = _data.reset_index()
# _data.to_csv('./data/dataset.csv', index=False)
# print(_data.columns.tolist())
# _data.head(10)